In [7]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import os

from pprint import pprint


from src.abtests import *
from src.utils import *
from src.pef_elasticity import *

pd.set_option('display.max_columns', None)
from datetime import *

import shap

spark = SparkSession.builder.getOrCreate()
register_spark_methods(spark)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
arpu_query = """
with revenue_customers as (
select
--     universities.id university_id
    universities.education_group_id
    ,kinds.parent_id as course_kind
    ,levels.parent_id as course_level
    ,scholar_semester(orders.registered_at) as scholar_semester
    ,count(distinct orders.id) n_orders
    ,count(distinct (case when orders.checkout_step='paid' and coupon_exchanges.id is null    
      then orders.id else null end)) n_paids
    ,round(coalesce(sum(case when orders.checkout_step='paid' and coupon_exchanges.id is null
      then pre_enrollment_fees.value
      +coalesce(split_rules.percentage/100,0)
      *offers.offered_price
      *(case when university_offers.max_payments<=1
          then 0.
        when university_offers.max_payments<=12
          then 9*(0.09884+0.0846312*(university_offers.max_payments-2)) 
        when university_offers.max_payments<=24
          then 9*(0.94515+0.0603743*(university_offers.max_payments-12)) 
        when university_offers.max_payments<=36
          then 9*(1.66965+0.0436079*(university_offers.max_payments-24)) 
        when university_offers.max_payments<=48
          then 9*(2.19294+0.0357802*(university_offers.max_payments-36)) 
        when university_offers.max_payments<=60
          then 9*(2.62230+0.0307322*(university_offers.max_payments-48))
        else offers.offered_price*2.69199 end)end),0),2) revenue
      ,count(distinct base_users.customer_id) customers

from querobolsa_production.orders
    join querobolsa_production.base_users on base_users.id=orders.base_user_id
    join querobolsa_production.line_items on line_items.order_id=orders.id
    join querobolsa_production.pre_enrollment_fees on pre_enrollment_fees.id=line_items.pre_enrollment_fee_id
    join querobolsa_production.offers on offers.id=line_items.offer_id
    join querobolsa_production.universities on universities.id=offers.university_id
    join querobolsa_production.university_offers on university_offers.id=offers.university_offer_id
    join querobolsa_production.courses on courses.id=offers.course_id
    left join (select * from querobolsa_production.kinds where parent_id is not null) kinds on kinds.name = courses.kind
    left join (select * from querobolsa_production.levels where parent_id is not null) levels on levels.name = courses.level
    left join querobolsa_production.coupons on coupons.order_id=orders.id
    left join querobolsa_production.refund_operations on refund_operations.coupon_id=coupons.id
    left join querobolsa_production.coupon_exchanges on coupon_exchanges.to_coupon_id=coupons.id
    left join billing_production.students on students.qb_id=orders.base_user_id
    left join billing_production.billing_courses on billing_courses.qb_course_id=courses.id
      and billing_courses.deleted_at IS NULL
    left join billing_production.billings on billings.billing_course_id=billing_courses.id
      and billings.student_id=students.id
    left join billing_production.split_rules on split_rules.business_rule_group_id=billings.business_rule_group_id
      and split_rules.recipient_id=213
      and split_rules.enabled
  where
    1=1
    and orders.checkout_step<>'initiated'

  
  and (
    ( (orders.registered_at > '2019-10-01') and (orders.registered_at < current_date()) )
    or ((orders.registered_at > '2018-10-01') and (orders.registered_at < (current_date() - interval 1 year)))
  )

    and kinds.parent_id in (8)
    and levels.parent_id in (1)
    and universities.education_group_id = 12

  group by 1,2,3,4
  order by 1
)
select 
  *
  ,revenue/customers arpu
from 
  revenue_customers
"""
df_arpu = spark.sql(arpu_query).toPandas()
df_arpu

AnalysisException: "expression 'querobolsa_production.orders.`registered_at`' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;"